In [1]:
import pandas as pd
import numpy as np

## 工序A

In [2]:
df = pd.read_excel('附件二.xlsx')
def transform_time(time):
    hour = time.hour
    minute = time.minute
    return hour * 60 + minute

df['数值型时间'] = df['到站时间'].apply(transform_time)

time_car_dict = {}
for time, car in zip(df['数值型时间'], df['动车类别']):
    time_car_dict[time] = car
    
time_rank_dict = {}
for time, rank in zip(df['数值型时间'], df['检修类型']):
    time_rank_dict[time] = rank
    
car_cost_A = {'CRH2': 60, "CRH3": 48 , "CRH5": 78 , "CRH6" : 60}

is_finish_A = np.zeros(shape = (len(df) ), dtype = int) # 记录所有动车的A工艺完成状态，1为已完成，0为未完成

m = len(df)

In [3]:
is_work_A = np.zeros(shape = (24*60 , 3), dtype = int)  ## 记录车间 [a1 ; a2 ; a3] 三个车间每分钟的工作状态。1为工作，0为空闲

input_time_record = []   ## 记录每个动车开始工序A的时间
finish_time_record = []  ## 记录每个动车完成工序A的时间
car_type_record = []     ## 按顺序记录每次检修动车类别
workshop_ind_record = [] ## 按顺序记录每次检修动车的车间
car_rank_record = [] ## 按顺序记录每次检修动车的车间


car_id = 0

for minute in range(len(is_work_A)):
    num_work_A = is_work_A[minute].sum()  ## 当前时间车间A的运作数量
    num_finish_A = is_finish_A.sum() ## 已经发出的机车数量
    if num_work_A == 0 and num_finish_A == m: ## 工艺完成
        break
    else:
        if num_work_A < 3 and minute in time_car_dict :## 有空闲车间 且 有新车进
            car_type = time_car_dict[minute]
            car_rank = time_rank_dict[minute]
            cost_time = car_cost_A[car_type]
            free_id = np.where(is_work_A[minute] == 0)[0][0]
            is_work_A[minute: minute + cost_time, free_id] = 1
            is_finish_A[car_id] = 1
            car_id += 1
            #print(car_id, minute + 1, car_type,free_id, cost_time )
            finish_time_record.append(minute + cost_time)
            car_type_record.append(car_type)
            input_time_record.append(minute)
            workshop_ind_record.append(free_id)
            car_rank_record.append(car_rank)
        
        if num_work_A == 3 and minute in time_car_dict :## 无空闲车间 且 有新车进，则推迟新车进来
            car_type = time_car_dict[minute]
            car_rank = time_rank_dict[minute]
            
            for k in range(10000):
                if minute + k not in time_car_dict:
                    time_car_dict[minute + k] = car_type
                    time_rank_dict[minute +k] = car_rank
                    break
            
            
df_A = pd.DataFrame({'动车类别':car_type_record, 
                     '检修等级': car_rank_record,
                  '进站时间': input_time_record,
                  '出站时间': finish_time_record,
                  '检修车间编号':workshop_ind_record})

df_A = df_A[['动车类别','检修等级','进站时间', '出站时间','检修车间编号']]

df_A['检修耗时(分钟)'] = df_A['出站时间'] - df_A['进站时间']

df_A['检修车间编号(展示)'] = df_A['检修车间编号'] + 1
df_A.to_csv('render.csv', encoding = 'gbk' , index = None)


## 工序B

In [4]:
## 检测等级为4的不需要经过工序B，应该先进行剔除 
mask = df_A['检修等级'] != 4
df = df_A[mask]

## 工序A的出战时间为工序B的进站时间
df = df[['动车类别', '检修等级', '出站时间']]
df.columns = ['动车类别','检修等级','工序b进站时间']

time_car_dict = {}
for time, car in zip(df['工序b进站时间'], df['动车类别']):
    time_car_dict[time] = car
    
time_rank_dict = {}
for time, rank in zip(df['工序b进站时间'], df['检修等级']):
    time_rank_dict[time] = rank
    
car_cost_B = {'CRH2': 120, "CRH3": 144 , "CRH5": 150 , "CRH6" : 162}

is_finish_B = np.zeros(shape = (len(df) ), dtype = int) # 记录所有动车的B工艺完成状态，1为已完成，0为未完成

m = len(df)

In [5]:
is_work_B = np.zeros(shape = (240*60 , 8), dtype = int)  ## 记录车间 [b1,b2,...,b8] 8个车间每分钟的工作状态。1为工作，0为空闲

input_time_record = []   ## 记录每个动车开始工序B的时间
finish_time_record = []  ## 记录每个动车完成工序B的时间
car_type_record = []     ## 按顺序记录每次检修动车类别
workshop_ind_record = [] ## 按顺序记录每次检修动车的车间
car_rank_record = []    ## 按顺序记录每次检修动车的车间


car_id = 0

for minute in range(len(is_work_B)):
    num_work_B = is_work_B[minute].sum()  ## 当前时间车间A的运作数量
    num_finish_B = is_finish_B.sum() ## 已经发出的机车数量
    if num_work_B == 0 and num_finish_B == m: ## 工艺完成
        break
    else:
        if num_work_B < 8 and minute in time_car_dict :## 有空闲车间 且 有新车进
            car_type = time_car_dict[minute]
            car_rank = time_rank_dict[minute]
            cost_time = car_cost_B[car_type]
            free_id = np.where(is_work_B[minute] == 0)[0][0]
            is_work_B[minute: minute + cost_time, free_id] = 1
            is_finish_B[car_id] = 1
            car_id += 1
            #print(car_id, minute + 1, car_type,free_id, cost_time )
            finish_time_record.append(minute + cost_time)
            car_type_record.append(car_type)
            input_time_record.append(minute)
            workshop_ind_record.append(free_id)
            car_rank_record.append(car_rank)
        
        if num_work_B == 8 and minute in time_car_dict :## 无空闲车间 且 有新车进，则推迟新车进来
            car_type = time_car_dict[minute]
            car_rank = time_rank_dict[minute]
            
            for k in range(10000):
                if minute + k not in time_car_dict:
                    time_car_dict[minute + k] = car_type
                    time_rank_dict[minute +k] = car_rank
                    break
            
            
df_B = pd.DataFrame({'动车类别':car_type_record, 
                     '检修等级': car_rank_record,
                  '进站时间': input_time_record,
                  '出站时间': finish_time_record,
                  '检修车间编号':workshop_ind_record})

df_B = df_B[['动车类别','检修等级','进站时间', '出站时间','检修车间编号']]

df_B['检修耗时(分钟)'] = df_B['出站时间'] - df_B['进站时间']

df_B['检修车间编号(展示)'] = df_B['检修车间编号'] + 1
df_B.to_csv('render.csv', encoding = 'gbk' , index = None)

## 工序C

In [6]:
## 检测等级为1、3的不需要经过工序C，应该先进行剔除 
mask = (df_B['检修等级'] == 1) | (df_B['检修等级'] == 3)
df = df_B[~mask]

## 检修等级为4的在工序B的时候剔除了，在工序C的时候应该补上
mask = df_A['检修等级'] == 4
data = df_A[mask]

df = pd.concat([df, data])

## 上一道工序的出战时间为下一道工序的进站时间
df = df[['动车类别', '检修等级', '出站时间']]
df.columns = ['动车类别','检修等级','工序c进站时间']

time_car_dict = {}
for time, car in zip(df['工序c进站时间'], df['动车类别']):
    time_car_dict[time] = car
    
time_rank_dict = {}
for time, rank in zip(df['工序c进站时间'], df['检修等级']):
    time_rank_dict[time] = rank
    
car_cost_C = {'CRH2': 90, "CRH3": 30 , "CRH5": 90 , "CRH6" : 18}

is_finish_C = np.zeros(shape = (len(df) ), dtype = int) # 记录所有动车的C工艺完成状态，1为已完成，0为未完成

m = len(df)

In [7]:
is_work_C = np.zeros(shape = (240*60 , 5), dtype = int)  ## 记录车间 [c1,c2,...,c5] 5个车间每分钟的工作状态。1为工作，0为空闲

input_time_record = []   ## 记录每个动车开始工序B的时间
finish_time_record = []  ## 记录每个动车完成工序B的时间
car_type_record = []     ## 按顺序记录每次检修动车类别
workshop_ind_record = [] ## 按顺序记录每次检修动车的车间
car_rank_record = []    ## 按顺序记录每次检修动车的车间


car_id = 0

for minute in range(len(is_work_C)):
    num_work_C = is_work_C[minute].sum()  ## 当前时间车间A的运作数量
    num_finish_C = is_finish_C.sum() ## 已经发出的机车数量
    if num_work_C == 0 and num_finish_C == m: ## 工艺完成
        break
    else:
        if num_work_C < 5 and minute in time_car_dict :## 有空闲车间 且 有新车进
            car_type = time_car_dict[minute]
            car_rank = time_rank_dict[minute]
            cost_time = car_cost_C[car_type]
            free_id = np.where(is_work_C[minute] == 0)[0][0]
            is_work_C[minute: minute + cost_time, free_id] = 1
            is_finish_C[car_id] = 1
            car_id += 1
            #print(car_id, minute + 1, car_type,free_id, cost_time )
            finish_time_record.append(minute + cost_time)
            car_type_record.append(car_type)
            input_time_record.append(minute)
            workshop_ind_record.append(free_id)
            car_rank_record.append(car_rank)
        
        if num_work_C == 5 and minute in time_car_dict :## 无空闲车间 且 有新车进，则推迟新车进来
            car_type = time_car_dict[minute]
            car_rank = time_rank_dict[minute]
            
            for k in range(10000):
                if minute + k not in time_car_dict:
                    time_car_dict[minute + k] = car_type
                    time_rank_dict[minute +k] = car_rank
                    break
            
            
df_C = pd.DataFrame({'动车类别':car_type_record, 
                     '检修等级': car_rank_record,
                  '进站时间': input_time_record,
                  '出站时间': finish_time_record,
                  '检修车间编号':workshop_ind_record})

df_C = df_C[['动车类别','检修等级','进站时间', '出站时间','检修车间编号']]

df_C['检修耗时(分钟)'] = df_C['出站时间'] - df_B['进站时间']

df_C['检修车间编号(展示)'] = df_C['检修车间编号'] + 1
df_C.to_csv('render.csv', encoding = 'gbk' , index = None)

## 工序D

In [8]:
## 检测等级为1、3的不需要经过工序C，应该先进行剔除 
mask = (df_C['检修等级'] == 1) | (df_C['检修等级'] == 2)
df = df_C[~mask]

## 检修等级为3的在工序C的时候剔除了，在工序D的时候应该补上
mask = df_B['检修等级'] == 3
data = df_B[mask]

df = pd.concat([df, data])

## 上一道工序的出战时间为下一道工序的进站时间
df = df[['动车类别', '检修等级', '出站时间']]
df.columns = ['动车类别','检修等级','工序d进站时间']

time_car_dict = {}
for time, car in zip(df['工序d进站时间'], df['动车类别']):
    time_car_dict[time] = car
    
time_rank_dict = {}
for time, rank in zip(df['工序d进站时间'], df['检修等级']):
    time_rank_dict[time] = rank
    
car_cost_D = {'CRH2': 240, "CRH3": 288 , "CRH5": 180 , "CRH6" : 300}

is_finish_D = np.zeros(shape = (len(df) ), dtype = int) # 记录所有动车的D工艺完成状态，1为已完成，0为未完成

m = len(df)

In [9]:
is_work_D = np.zeros(shape = (240*60 , 3), dtype = int)  ## 记录车间 [d1,d2,d3] 3个车间每分钟的工作状态。1为工作，0为空闲

input_time_record = []   ## 记录每个动车开始工序B的时间
finish_time_record = []  ## 记录每个动车完成工序B的时间
car_type_record = []     ## 按顺序记录每次检修动车类别
workshop_ind_record = [] ## 按顺序记录每次检修动车的车间
car_rank_record = []    ## 按顺序记录每次检修动车的车间


car_id = 0

for minute in range(len(is_work_D)):
    num_work_D = is_work_D[minute].sum()  ## 当前时间车间A的运作数量
    num_finish_D = is_finish_D.sum() ## 已经发出的机车数量
    if num_work_D == 0 and num_finish_D == m: ## 工艺完成
        break
    else:
        if num_work_D < 3 and minute in time_car_dict :## 有空闲车间 且 有新车进
            car_type = time_car_dict[minute]
            car_rank = time_rank_dict[minute]
            cost_time = car_cost_D[car_type]
            free_id = np.where(is_work_D[minute] == 0)[0][0]
            is_work_D[minute: minute + cost_time, free_id] = 1
            is_finish_D[car_id] = 1
            car_id += 1
            #print(car_id, minute + 1, car_type,free_id, cost_time )
            finish_time_record.append(minute + cost_time)
            car_type_record.append(car_type)
            input_time_record.append(minute)
            workshop_ind_record.append(free_id)
            car_rank_record.append(car_rank)
        
        if num_work_D == 3 and minute in time_car_dict :## 无空闲车间 且 有新车进，则推迟新车进来
            car_type = time_car_dict[minute]
            car_rank = time_rank_dict[minute]
            
            for k in range(10000):
                if minute + k not in time_car_dict:
                    time_car_dict[minute + k] = car_type
                    time_rank_dict[minute +k] = car_rank
                    break
            
            
df_D = pd.DataFrame({'动车类别':car_type_record, 
                     '检修等级': car_rank_record,
                  '进站时间': input_time_record,
                  '出站时间': finish_time_record,
                  '检修车间编号':workshop_ind_record})

df_D = df_D[['动车类别','检修等级','进站时间', '出站时间','检修车间编号']]

df_D['检修耗时(分钟)'] = df_D['出站时间'] - df_D['进站时间']

df_D['检修车间编号(展示)'] = df_D['检修车间编号'] + 1
df_D.to_csv('render.csv', encoding = 'gbk' , index = None)

## 工艺E

In [10]:
## 只需要检修等级为4、5的
mask = (df_D['检修等级'] == 4) | (df_D['检修等级'] == 5)
df = df_D[mask]

## 上一道工序的出战时间为下一道工序的进站时间
df = df[['动车类别', '检修等级', '出站时间']]
df.columns = ['动车类别','检修等级','工序e进站时间']

time_car_dict = {}
for time, car in zip(df['工序e进站时间'], df['动车类别']):
    time_car_dict[time] = car
    
time_rank_dict = {}
for time, rank in zip(df['工序e进站时间'], df['检修等级']):
    time_rank_dict[time] = rank
    
car_cost_E = {'CRH2': 420, "CRH3": 390 , "CRH5": 390 , "CRH6" : 420}

is_finish_E = np.zeros(shape = (len(df) ), dtype = int) # 记录所有动车的D工艺完成状态，1为已完成，0为未完成

m = len(df)

In [11]:
is_work_E = np.zeros(shape = (240*60 , 2), dtype = int)  ## 记录车间 [e1,e2] 2个车间每分钟的工作状态。1为工作，0为空闲

input_time_record = []   ## 记录每个动车开始工序E的时间
finish_time_record = []  ## 记录每个动车完成工序E的时间
car_type_record = []     ## 按顺序记录每次检修动车类别
workshop_ind_record = [] ## 按顺序记录每次检修动车的车间
car_rank_record = []    ## 按顺序记录每次检修动车的车间


car_id = 0

for minute in range(len(is_work_E)):
    num_work_E = is_work_E[minute].sum()  ## 当前时间车间A的运作数量
    num_finish_E = is_finish_E.sum() ## 已经发出的机车数量
    if num_work_E == 0 and num_finish_E == m: ## 工艺完成
        break
    else:
        if num_work_D < 2 and minute in time_car_dict :## 有空闲车间 且 有新车进
            car_type = time_car_dict[minute]
            car_rank = time_rank_dict[minute]
            cost_time = car_cost_E[car_type]
            free_id = np.where(is_work_E[minute] == 0)[0][0]
            is_work_E[minute: minute + cost_time, free_id] = 1
            is_finish_E[car_id] = 1
            car_id += 1
            #print(car_id, minute + 1, car_type,free_id, cost_time )
            finish_time_record.append(minute + cost_time)
            car_type_record.append(car_type)
            input_time_record.append(minute)
            workshop_ind_record.append(free_id)
            car_rank_record.append(car_rank)
        
        if num_work_E == 2 and minute in time_car_dict :## 无空闲车间 且 有新车进，则推迟新车进来
            car_type = time_car_dict[minute]
            car_rank = time_rank_dict[minute]
            
            for k in range(10000):
                if minute + k not in time_car_dict:
                    time_car_dict[minute + k] = car_type
                    time_rank_dict[minute +k] = car_rank
                    break
            
            
df_E = pd.DataFrame({'动车类别':car_type_record, 
                     '检修等级': car_rank_record,
                  '进站时间': input_time_record,
                  '出站时间': finish_time_record,
                  '检修车间编号':workshop_ind_record})

df_E = df_E[['动车类别','检修等级','进站时间', '出站时间','检修车间编号']]

df_E['检修耗时(分钟)'] = df_E['出站时间'] - df_E['进站时间']

df_E['检修车间编号(展示)'] = df_E['检修车间编号'] + 1
df_E.to_csv('render.csv', encoding = 'gbk' , index = None)

### 计算总耗时

In [12]:
(df_E['出站时间'].max() - df_A['进站时间'].min() ) / 60 ## 小时

18.75